# Tuning

The model that I have received in the previous notebook is pretty good, but perhaps it's not at its best still. 
I will be decomposing the voted prediction of the forest to each individual tree's vote to look at the predictions, and arrange them into a matrix. This will allow me to peek just a little more into our models, and make them even better 


In [2]:
#Im going to steal my professors datascience libraries off of dropbox
import os
home_path =  os.path.expanduser('~')
os.chdir(home_path + '/Dropbox/cis399_ds1_f17/week_libraries/datascience_1')
!git pull

WindowsError: [Error 3] The system cannot find the path specified: 'C:\\Users\\Snapu/Dropbox/cis399_ds1_f17/week_libraries/datascience_1'

In [3]:
import pandas as pd
import sys

url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSVp-qDGOnClv6b4pdr2OrHZ3lmmvk0zafSAJ0v-uo69xw2Qqw26Xk8cNc8u-hviteApCzJn0ErKdMT/pub?gid=1702417835&single=true&output=csv'
skate_table = pd.read_csv(url)

pd.__version__  # should see 0.20.3 or higher

#This is the local file of my functions 

sys.path.append(home_path + '\Documents\CIS 399\datascience_1')
from week7 import *
%who function



accuracy	 build_pred	 build_tree_iter	 caser	 closest_centroid	 compute_mean	 compute_prediction	 compute_training	 euclidean_distance	 
f1	 find_best_splitter	 forest_builder	 generate_table	 gig	 gini	 informedness	 initialize_centroids	 k_fold	 
k_means	 phase_1	 phase_2	 predictor_case	 probabilities	 row_to_vect	 seeder	 tree_predictor	 vote_taker	 



In [4]:
pd.set_option('display.max_columns', None)
skate_table.head(10)

,Season,Series,City,Country,Year,Month,Day,Distance,Round,Group,Num_Skater,Name,Nationality,Rank_In_Group,Start_Position,Time,Qualification,rank_lap1,time_lap1,rank_lap2,time_lap2,rank_lap3,time_lap3,rank_lap4,time_lap4,rank_lap5,time_lap5,kmeans_time_lap5,filled_time,winning,city_Calgary,city_Debrecen,city_Dordrecht,city_Dresden,city_Erzurum,city_Gangneung,city_Kolomna,city_Malmo,city_Montreal,city_Moscow,city_Nagoya,city_SaltLakeCity,city_Seoul,city_Shanghai,city_Sochi,city_Torino,city_Toronto,city_nan,round_FinalA,round_FinalB,round_Heats,round_PrePreliminaries,round_Preliminaries,round_Quarterfinals,round_RankingRaces,round_RepHeats,round_RepQuarterfinals,round_RepSemifinals,round_Semifinals,round_nan,year_2012.0,year_2013.0,year_2014.0,year_2015.0,year_2016.0,year_nan,nat_ARG,nat_AUS,nat_AUT,nat_BEL,nat_BIH,nat_BLR,nat_BUL,nat_CAN,nat_CHN,nat_COL,nat_CRO,nat_CZE,nat_DEN,nat_ESP,nat_FRA,nat_GBR,nat_GER,nat_HKG,nat_HUN,nat_IND,nat_IRL,nat_ISR,nat_ITA,nat_JPN,nat_KAZ,nat_KOR,nat_LAT,nat_LUX,nat_MAS,nat_NED,nat_NOR,nat_NZL,nat_POL,nat_PRK,nat_ROU,nat_RUS,nat_SGP,nat_SLO,nat_SRB,nat_SUI,nat_SVK,nat_SWE,nat_THA,nat_TPE,nat_TUR,nat_UKR,nat_USA,nat_nan,time_lap1_par,time_lap2_par,time_lap3_par,time_lap4_par,time_lap5_par,forest_1,forest_1_type,forest_2,forest_2_type,forest_3,forest_3_type
0,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,1,153,AN Victor',RUS,1,1,41.311,Q,1,6.94,1,8.70,1,8.37,1,8.53,1,8.78,8.78,41.311,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,true_negative,0,true_negative,0,true_negative
1,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,1,114,SHI Jingnan',CHN,2,3,41.358,Q,2,7.12,2,8.68,2,8.27,2,8.56,2,8.73,8.73,41.358,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,true_negative,0,true_negative,0,true_negative
2,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,1,129,David Mano LOTH',HUN,3,2,43.338,?,3,7.15,3,9.30,3,8.76,3,8.93,3,9.2,9.20,43.338,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,true_negative,0,true_negative,0,true_negative
3,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,2,155,Vladimir GRIGOREV',RUS,1,3,40.984,Q,2,7.03,2,8.85,2,8.24,2,8.37,1,8.5,8.50,40.984,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,false_negative,0,false_negative,0,false_negative
4,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,2,165,J R CELSKI',USA,2,1,40.992,Q,1,6.88,1,8.87,1,8.29,1,8.36,2,8.6,8.60,40.992,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,false_negative,0,false_negative,0,false_negative
5,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,2,103,Nathaniel HENRY',AUS,3,2,42.665,?,3,7.04,3,9.11,3,8.66,3,8.88,3,8.98,8.98,42.665,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,true_negative,0,true_negative,0,true_negative
6,2013/2014,World Championships',Montreal,CAN,2014,3,14,500m,Preliminaries,2,186,Yonaton Spencer SUBIN',ISR,4,4,47.282,?,4,7.89,4,9.97,4,9.74,4,9.75,4,9.94,9.94,47.282,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,true_negative

In [28]:
splitter_columns = [
       'city_Calgary',
       'city_Debrecen', 'city_Dordrecht', 'city_Dresden', 'city_Erzurum',
       'city_Gangneung', 'city_Kolomna', 'city_Malmo', 'city_Montreal',
       'city_Moscow', 'city_Nagoya', 'city_SaltLakeCity', 'city_Seoul',
       'city_Shanghai', 'city_Sochi', 'city_Torino', 'city_Toronto',
       'city_nan', 'round_FinalA', 'round_FinalB', 'round_Heats',
       'round_PrePreliminaries', 'round_Preliminaries',
       'round_Quarterfinals', 'round_RankingRaces', 'round_RepHeats',
       'round_RepQuarterfinals', 'round_RepSemifinals',
       'round_Semifinals', 'round_nan', 'year_2012.0', 'year_2013.0',
       'year_2014.0', 'year_2015.0', 'year_2016.0', 'year_nan',
       'time_lap1_par', 'time_lap2_par', 'time_lap3_par', 'time_lap4_par',
       'time_lap5_par']

In [29]:
def oob_testing(forest, table, target):
    oobs = set()
    for i in range(len(forest)):
        temp = set(forest[i]['oob'])
        oobs = oobs.union(temp)
    left_out = table.loc[table.index.isin(oobs)]
    forest_oob = build_tree_iter(table, splitter_columns, target)
    return caser(left_out, forest_oob, target)

forest1 = forest_builder(skate_table, splitter_columns, 'winning', hypers={'total-trees':5})
forest1_oob_types = oob_testing(forest1, skate_table, 'winning')
print(forest1_oob_types.sum())  # length of testing table
print((accuracy(forest1_oob_types), f1(forest1_oob_types), informedness(forest1_oob_types)))

4301
(0.9416414787258777, 0.38329238329238335, 0.2540007745868451)


## Pruning

We are looking at the outcome of each of the trees of the forest to see if it predicted a time under 41 seconds.

We should prune out the trees which are not very helpful to us. Well, if we think about a forest in which all the trees are all saying one thing, then it always predicts a time greater than 41 or one under 41. That's not very useful information. Basically, these models are saying that times under 41 or over 41 are impossible depending on whether they are all 1's or 0's.

### Trivial Rows
I will use the prediction matrix to determine rows that are not useful to our model, as being ones, which have uniform entries across the rows.

I will preface this part with knowing that I'm not 100% certain of the effect of thinning the data on how it predicts the data. So, I will use the effect of how it tests against its own training data and from the original table. Either way you will see it yields something interesting.


In [30]:
all_votes = []
def vote_matrix(row,forest):
    row_pred = []
    for tree in forest:
        prediction = tree_predictor(row, tree)
        row_pred.append(prediction)
    all_votes.append(row_pred)

skate_table.apply(lambda row: vote_matrix(row, forest1), axis=1)

def non_trivial_rows(all_votes):
    t_row_indicies = []
    i = 0
    for row in all_votes:
        unan_count = 0
        for element in row:
            unan_count += element
        if (unan_count != 0) and (unan_count != len(row)):
            t_row_indicies.append(i)
        i += 1
    return t_row_indicies
            

In [31]:
nt_rows = non_trivial_rows(all_votes)
nt_skate_table = skate_table.loc[skate_table.index.isin(nt_rows)]
print(len(nt_skate_table))

116


In [32]:
forest2 = forest_builder(nt_skate_table, splitter_columns, 'winning', hypers={'total-trees':5})
forest2_oob_types = oob_testing(forest2, nt_skate_table, 'winning')
print("Self-Test:")
print(forest2_oob_types.sum())  # length of testing table
print((accuracy(forest2_oob_types), f1(forest2_oob_types), informedness(forest2_oob_types)))
print("Original-Data:")
forest2_oob_types_original = oob_testing(forest2, skate_table, 'winning')
print(forest2_oob_types_original.sum())  # length of testing table
print((accuracy(forest2_oob_types_original), f1(forest2_oob_types_original), informedness(forest2_oob_types_original)))

Self-Test:
109
(0.7522935779816514, 0.7969924812030076, 0.47185998627316406)
Original-Data:
109
(0.7339449541284404, 0.7943262411347517, 0.41386410432395326)


The length of the new table which we are making the model from is a small number compared to the original data set. This is alright because we expect about 7% of the data to be times under 41. 

We then build a random forest using this new table that has non- trivial rows. This should correspond to modeling more of the actual data, instead of the noise. 

As we can see this new forest performs better on f1 and informedness, but losses in accuracy. The f1 and informedness score take into account the accuracy of the model, and are telling us that this low accuracy is alright to have because our model is getting more precise and having better recall, than before.


### Trivial Trees

I will look at the sums of each of the columns of the rows to see how the distribution of the trees predictions are working. Perhaps some trees are doing most of the predictions, and others are predicting nothing. If a tree is predicting only 1 thing, then maybe this is a bad tree, and we need to do something with that bad tree.


In [33]:
def never_change(all_votes):
    constants = []
    for i in range(len(all_votes[0])): # we can do this because it's a matrix
        transposed = []                # so we can sum the columns
        column_counter = 0
        for row in all_votes:
            transposed.append(row[i])
        for element in transposed:
            column_counter += element
        if (column_counter == 0) or (column_counter == len(all_votes)):
            constants.append(i)
    constants.reverse()
    return constants

This algorithm is going to use the transpose of the matrix and then sum the rows to see if the trees are only voting one way.
It takes in the vote matrix, then returns the index of the trees that are trivial.

In [34]:
constants = never_change(all_votes)
forest3 = forest1
for constant in constants:
    del forest3[constant]



In [35]:
forest3_oob_types = oob_testing(forest3, skate_table, 'winning')
print(forest3_oob_types.sum())  # length of testing table
print((accuracy(forest3_oob_types), f1(forest3_oob_types), informedness(forest3_oob_types)))

3585
(0.9414225941422594, 0.35975609756097565, 0.2350185813182022)


The removal of these trivial trees seems to reduce our prediction power. This isn't good, because now we care going from a forest of 5 trees to one with less. The loss of trees and prediction power is not good, so we need to think of a way that we can raise the prediction power of our model, before I try and improve the model, I will see if the same holds for the model that we got from removing the trivial rows of the dataset. 

I will be removing the trivial trees and looking at the effect of this move. 

In [36]:
nt_skate_table.apply(lambda row: vote_matrix(row, forest2), axis=1)
constants2 = never_change(all_votes)
for constant in constants:
    del forest2[constant]
forest2_oob_types = oob_testing(forest2, nt_skate_table, 'winning')
print("Self-Test:")
print(forest2_oob_types.sum())  # length of testing table
print((accuracy(forest2_oob_types), f1(forest2_oob_types), informedness(forest2_oob_types)))
print("Original-Data:")
forest2_oob_types_original = oob_testing(forest2, skate_table, 'winning')
print(forest2_oob_types_original.sum())  # length of testing table
print((accuracy(forest2_oob_types_original), f1(forest2_oob_types_original), informedness(forest2_oob_types_original)))

Self-Test:
94
(0.7446808510638298, 0.7818181818181817, 0.4697802197802199)
Original-Data:
94
(0.723404255319149, 0.7796610169491526, 0.4084249084249083)


As we expect this also reduced the prediction power of the model. I will now try and find a way to raise the prediction power of the model by trying to get back to the normal size of the forest. 

## Addition

The removal of the trees leads to a decrease in our metrics, because we are losing predicting power even if it is negligible. So, I think it would be best to get more trees that aren't trivial to increase the prediction power. To get more negligible trees I will generate another random forest and add them into the original forest model until none of the trees are trivial anymore. I automated this process so we can check the different models I have generated to see if it a consistent move and one that is going to increase our metrics.


In [37]:
all_votes = []
forest4 = forest_builder(skate_table, splitter_columns, 'winning', hypers={'total-trees':5})
def new_trees(constants, forest3, forest4):
    for i in constants:
        forest3.append(forest4[i])
    skate_table.apply(lambda row: vote_matrix(row, forest3), axis=1)
    constants = never_change(all_votes)
    print(constants)
    if len(constants) > 0:
        for i in constants:
            del forest3[i]
            forest4 = forest_builder(skate_table, splitter_columns, 'winning', hypers={'total-trees':5})
            forest3 = new_trees(constants, forest3, forest4)
    return forest3
    



forest3 = new_trees(constants, forest3, forest4)
forest3_oob_types = oob_testing(forest3, skate_table, 'winning')
print(forest3_oob_types.sum())  # length of testing table
print((accuracy(forest3_oob_types), f1(forest3_oob_types), informedness(forest3_oob_types)))

[4, 3]
[3]
[]
[]
4023
(0.9415858811831966, 0.36997319034852555, 0.24382268361587434)


Nice, it appears that we have increased the prediction power slightly. This is a nice tuning for the model. However we had to go through multiple trees iterations. This addition of trees is raising our prediction power in a positive way. The model gained score across all metrics. It seems this move was pretty worthwhile.

I will see how the addition of the non-trivial rows model trees being added to the non-trivial model. I chose to get it from the nt_skate_table, and not the original table, because I want all my trees to be from the same data in my forest.  


In [38]:
all_votes = []
forest4 = forest_builder(nt_skate_table, splitter_columns, 'winning', hypers={'total-trees':5})
forest5 = new_trees(constants, forest2, forest4)
forest5_oob_types = oob_testing(forest5, nt_skate_table, 'winning')
print("Self-Test:")
print(forest5_oob_types.sum())  # length of testing table
print((accuracy(forest5_oob_types), f1(forest5_oob_types), informedness(forest5_oob_types)))
print("Original-Data:")
forest5_oob_types_original = oob_testing(forest5, skate_table, 'winning')
print(forest5_oob_types_original.sum())  # length of testing table
print((accuracy(forest5_oob_types_original), f1(forest5_oob_types_original), informedness(forest5_oob_types_original)))

[4]
[]
Self-Test:
105
(0.7714285714285715, 0.8125, 0.5115499254843516)
Original-Data:
1833
(0.9328968903436988, 0.5138339920948617, 0.3838029462547299)


We got the informedness above .5 using the addition of the trees for the self-test. However, the production for the original data went down. This seems like we are overfitting the nt_skate_table. I am going to be removing the tree we added and I'm going to exchange it for one that is from the original skate table and see if this is going to cause a fit to the original table to increase and the non-trivial table fit to go down

In [41]:
del forest5[4]
forest4 = forest_builder(skate_table, splitter_columns, 'winning', hypers={'total-trees':5})
forest5.append(forest4[0])
skate_table.apply(lambda row: vote_matrix(row, forest5), axis=1)
print(never_change(all_votes))

forest5_oob_types = oob_testing(forest5, nt_skate_table, 'winning')
print("Self-Test:")
print(forest5_oob_types.sum())  # length of testing table
print((accuracy(forest5_oob_types), f1(forest5_oob_types), informedness(forest5_oob_types)))
print("Original-Data:")
forest5_oob_types_original = oob_testing(forest5, skate_table, 'winning')
print(forest5_oob_types_original.sum())  # length of testing table
print((accuracy(forest5_oob_types_original), f1(forest5_oob_types_original), informedness(forest5_oob_types_original)))

[]
Self-Test:
107
(0.7476635514018691, 0.7938931297709924, 0.46115466856735576)
Original-Data:
1818
(0.935093509350935, 0.5163934426229507, 0.39081721723275065)


We got the informedness above .5 using the addition of the trees for the self-test. However, the production for the original data went down. This seems like we are overfitting the nt_skate_table. I am going to be removing the tree we added and I'm going to exchange it for one that is from the original skate table and see if this is going to cause a fit to the original table to increase and the non-trivial table fit to go down.

I will choose the Non-Trivial Rows Model because this has the highest informedness score and f1 score for the prediction of the original table.